# Lab 7: Controls Part 1
### EECS 16B: Designing Information Devices and Systems II, Summer 2022

Updated by Mingyang Wang (2022)

quackquack@berkeley.edu

Updated by Yi-Hsuan Shih (2021, 2022)

yihaun@berkeley.edu

Hossein Najafi (2021)

hossein_najafi@berkeley.edu

## Table of Contents

* [Introduction](#part0)
* [Part 1: Open-Loop Control](#part1)
    * [Part 1.1: Open-Loop Controller Design](#part11)
    * [Part 1.2: Open-Loop Simulation](#part12)
    * [Part 1.3: Jolt Calculation](#part13)
    * [Part 1.4:  Open-Loop Implementation](#part13)
* [Part 2: Closed-Loop Control](#part2)
    * [Part 2.1: Closed-Loop Design](#part21)
    * [Part 2.2: Closed-Loop Simulation and Feedback Gain Selection](#part22)
    * [Part 2.3: Closed-Loop Implementation](#part23)
    * [Part 2.4: Feedback Gain Tuning](#part24)
    * [Part 2.5: Steady-State Error Correction](#part25)
 
## <span style="color:#ba190f"> You need to run the Python scripts one by one, or errors will show up as they rely on the variables defined above!!

## <span style="color:#ba190f"> DO NOT include units when submitting your answers on Gradescope! ONLY include the numerical value rounded to the number of decimal places specified in each question, in units specified in the question. DO NOT include letters, words, etc. If a question involves entering an answer that is not a numerical value, the format of the answer will be clearly specified in that question.

<a id='part0'></a>
# <span style="color:navy">Part 0: Introduction</span>
In System ID lab you modeled the open-loop system, collected data, and determined the unknown parameters of your car.

This week, you'll implement the open-loop and closed-loop systems. By the end of the phase, you should be able to control the trajectory of the car to go straight. We will return to turning later in the following lab next week.

The goals of this phase are as follows:
- Open-loop controller design
- Closed-loop controller design 



## Lab Notes
- [Main Lab Note](https://drive.google.com/file/d/1zOL1_qXV2KM-Sc727ymCfZUzt4oqIgoE/view?usp=sharing)
- [Lab Note 6: System ID](https://drive.google.com/file/d/1J4KrMvA9uT2QnrG0sPYQ2maG9munTxnX/view?usp=sharing)
- Optional: [Extra resource](https://inst.eecs.berkeley.edu/~ee16b/fa17/proj/controls-primer.pdf) (The derivation of the closed-loop control system and its eigenvalue)

Review the lab note. The lab note provides a technical review of the concepts covered in lab as well as theoretical connections. **You should read the lab notes before your lab section.** They serve as a review and will be helpful as a reference during section.



## Pre-Lab

### Complete the pre-lab assignment before doing the lab. For all students, submit your answers to the Gradescope assignment "Pre-Lab 7: Controls Part 1". Pre-Lab 7 is due on Tuesday, July 19 at 11:59pm. No late submissions will be accepted.


<a id='part1'></a>
# <span style="color:navy">Part 1: Open-Loop Control</span>

## 1.0 Model Parameters from Lab 6: System ID


### <a href="https://drive.google.com/file/d/1zOL1_qXV2KM-Sc727ymCfZUzt4oqIgoE/view?usp=sharing">Read Part 1 of the lab note to familiarize yourself with/remind yourself of the open-loop model.</a>

### <span style="color:red">**Use the System ID TinkerCAD modules for this lab.**</span>

### <span style="color:red">**Fill in your model parameters and target velocity with values you calculated at the end of System ID lab.**</span>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import utils
%matplotlib inline

In [2]:
# YOUR PARAMETERS HERE
theta_left = ...;
theta_right = ...;
beta_left = ...;
beta_right = ...;
v_star = ...;


simulator = utils.Simulator(theta_left, theta_right, beta_left, beta_right)

In [3]:
params = np.array([(theta_left, theta_right),
                   (beta_left, beta_right)])
d0 = (0,  0)
sim_length = 20 # sim length
mismatch_error = 0.05 # 5% model mismatch beta^* = beta*(1+mismatch_error), theta^* = theta*(1+mismatch_error)

**<span style="color:#075a04">1. What is the value of theta_left? Enter a numerical value with two decimal places (e.g. 3.14).**
    
< YOUR ANSWER ON GRADESCOPE >

**<span style="color:#075a04">2. What is the value of theta_right? Enter a numerical value with two decimal places (e.g. 3.14).**
    
< YOUR ANSWER ON GRADESCOPE >

**<span style="color:#075a04">3. What is the value of beta_left? Enter a numerical value with two decimal places (e.g. 3.14).**
    
< YOUR ANSWER ON GRADESCOPE >

**<span style="color:#075a04">4. What is the value of beta_right? Enter a numerical value with two decimal places (e.g. 3.14).**
    
< YOUR ANSWER ON GRADESCOPE >

**<span style="color:#075a04">5. What is the value of v_star? Enter a numerical value with two decimal places (e.g. 3.14).**
    
< YOUR ANSWER ON GRADESCOPE >

<a id='part11'></a>
## 1.1 Open-Loop Controller Design

Now design an open-loop controller that uses the desired wheel velocity (the operation point you chose in System ID lab), `v_star`, to set the input `u` to an appropriate value for the pulse width modulation (PWM). To do this, solve your model from the previous part for the input $u[i]$ that makes the model velocity $d[i+1] - d[i]$ equal to the desired velocity $v^*$.

In [4]:
# DESIGN YOUR OPEN-LOOP CONTROL HERE
def drive_straight_left_ol(v_star):
    """Returns the open loop input 'u' for the left wheel"""
    # YOUR CODE HERE
    return # u

def drive_straight_right_ol(v_star):
    """Returns the open loop input 'u' for the right wheel"""
    # YOUR CODE HERE
    return # u

<a id='part12'></a>
## 1.2 Open-Loop Simulation

Now, let's do a quick simulation to check the behavior of the car based on our functions above. In these simulations, we apply maximum PWM for a short time as we do in the real car to "jolt" the motors from a stop. Since we deliberately used different resistor value for right and left wheels in our front-end circuit, it is expected that the initial velocity is not the same for left and right wheels and thus $\delta[1] \neq 0$.

**Important:** As long as $\delta[i]$ is constant, even if $\delta[i]\neq 0$, the car still travels straight! Traveling straight means the *velocities*, not the positions, of the wheels are the same. It may just travel straight in a different direction than the original heading if $\delta[i]\neq 0$. Lets call the steady state (final) value $\delta[i]\overset{i\rightarrow \infty}{=}\delta_{ss}$.

## Mismatch
In the System ID lab, we found $\beta_{L,R}$ and $\theta_{L,R}$ to model two wheels. However, there could be a mismatch between the estimated model parameters $\beta_{L,R}, \theta_{L,R}$ and real model parameters $\beta^*_{L,R}, \theta^*_{L,R}$. For example, the nonideality of the PWM signal from the Arduino can cause the mismatch of the model parameters. In this simulation, we assumed that the mismatch error is the same for both $\beta_{L,R}$ and $\theta_{L,R}$

$$\theta^*_{L,R}=\theta_{L,R} \times (1+\text{mismatch\_error})$$
$$\beta^*_{L,R}=\beta_{L,R} \times (1+\text{mismatch\_error})$$


For the first 4 graphs, in the top two plots, we assume the physical system perfectly matches our model. At the bottom, we assume the physical system is 5% different from our model, meaning the model parameters of the wheels have a 5% mismatch (mismatch_error = 0.05).

The 5th plot titled "Trajectory of the car" shows how the car would travel depending on which model is used. The car should ideally be traveling straight, but model mismatch often introduce deviations from the ideal case.

In [ ]:
titles = ["Open-loop control with perfect model", "Open-loop control with model mismatch"]

utils.two_sims(titles, simulator, v_star, drive_straight_left_ol, drive_straight_right_ol,sim_length=sim_length)
utils.plot_car_traj(titles, simulator, v_star, drive_straight_left_ol, drive_straight_right_ol,mismatch_error_arr=[0, mismatch_error],sim_length=sim_length,  u_noise_factor_arr=[0, 0])

From the above plots, we can see that the open-loop car travels straight only with a perfect model.

**<span style="color:#075a04">6. What is $\delta_{ss}$ for an ideal model? Only enter the numerical value with two decimal places (e.g. 3.14) in units shown on the plots. If $\delta_{ss}$ is unbounded, enter -1.</span>**

< YOUR ANSWER ON GRADESCOPE >

**<span style="color:#075a04">7. What is $\delta_{ss}$ for a model with 5% mismatch? Only enter the numerical value  with two decimal places (e.g. 3.14) in units shown on the plots. If $\delta_{ss}$ is unbounded, enter -1. </span>**

< YOUR ANSWER ON GRADESCOPE >

**<span style="color:#075a04">8. Observe the trajectory of the car in the two cases. Does a 5% mismatch introduce any deviation in the car's path? If so, please select the option best describing what you see. </span>**

< YOUR ANSWER ON GRADESCOPE >
* A. The car zigzags around the line y = 0 while moving forward.
* B. The car moves in circles.
* C. The car goes left and right along the x axis (horizontally).
* D. The car goes up and down along the y axis (vertically).
* E. The car does not experience any deviation from the ideal path.

## Noise
In the real world, there is also noise being introduced by various sources, further complicating the imperfections of the open loop control module. Noise sources that can have significant impact on the system could be electronical noise, mechanical noise, measurement noise, and quantization noise. To better visualize the effect of noise in the control of the car, we will deliberately inject (add) noise into the data we measured to emulate real-life measurements. We have added a random discrete uniform noise ($w_u$) to the inputs ($u$) of a magnitude equal to 5% of the total amplitude. Feel free to experiment with other values and see what happens. To do so, change the value of the `u_noise_factor` variable in the noisy model instantiation. (a value of 0.05 means 5%, and so on)

$$u[i]^* = u[i] + w_u[i]$$

where $0\leq u[i] \leq 225, |w_u[i]| < 0.5 \times \text{u\_noise\_factor} \times 225, \text{mean}(w_u[i])=0  $

In [ ]:
titles = ["Open-loop control with perfect model",
          "Open-loop control with model mismatch only",
          "Open-loop control with model mismatch + noise"]

## Simulating perfect, mismatch, mismatch + noise model
utils.plot_car_traj(titles, simulator, v_star, drive_straight_left_ol, drive_straight_right_ol,mismatch_error_arr=[0, mismatch_error, mismatch_error],sim_length=sim_length,u_noise_factor_arr=[0.0, 0.0, 0.05])

**<span style="color:#075a04">9. Observe the trajectory of the car in the two cases with mismatch in the model (with and without noise). Select *all* the options that best describe what you see. </span>**

< YOUR ANSWER ON GRADESCOPE >

* A. The trajectory is no longer the same as the noiseless case.
* B. The car trajectory is no longer deterministic and changes with every run.
* C. The car fails to move when more than 25% noise is added.
* D. The trajectory becomes closer to a circle as u_noise_factor decreases.
* E. The effects of noise on the system are very minimal and cannot be visualized.

<a id='part13'></a>
## 1.3 Jolt Calculation


We find the jolts below by calculating the inputs that result in the minimum of the maximum velocities of both wheels. 

- **<span style="color:red">Copy your `data.txt` file from System ID to the directory where iPython notebook locates.</span>**
- Run the code below to find your initial jolt values.
- Your initial jolt values are correct if they are the input values that correspond to the points where the blue and yellow lines intersect the green line respectively.

In [ ]:
left_jolt, right_jolt = utils.find_jolt()
print('int left_jolt = {};\nint right_jolt = {};'.format(left_jolt, right_jolt))

**<span style="color:#075a04">10. What is the value for left_jolt? Enter the numerical value as displayed by the script. (should be an integer, e.g. 100)</span>**

< YOUR ANSWER ON GRADESCOPE >

**<span style="color:#075a04">11. What is the value for right_jolt? Enter the numerical value as displayed by the script.(should be an integer, e.g. 100) </span>** 

< YOUR ANSWER ON GRADESCOPE >


##  Now open your TinkerCAD module.


-----

<a id='part14'></a>
## 1.4 Open-Loop Implementation

Now implement the controller on the Arduino. The sketch **`open_loop.ino`** contains some code that implements the open loop control.

- Fill in these values in `CODE BLOCK CON1`:
    - desired velocity: `v_star` 
    - estimated parameters: `theta_left`, `theta_right`, `beta_left`, `beta_right`
    - initial jolt values: `left_jolt`, `right_jolt`
    
    
- Additionally, we need to compute the PWM input to the motors in `CODE BLOCK CON2`:
    - Complete `drive_straight_left()` and `drive_straight_right()` to compute `left_cur_pwm` and `right_cur_pwm` (the PWM inputs for each wheel required for a desired v_star)
    
    
- Fill out `CODE BLOCK CON0` to calculate the control inputs using the functions from `CODE BLOCK CON2`.
- Upload the sketch to your Arduino.

This code will attempt at driving both wheels at the same speed so the car remains on a straight path for some period of time. Because of the asymmetry we have introduced using the different base resistances, the left and right motor signals will not be identical. 
- Observe these voltages by probing the voltage at the base of each transistor. Make sure you confirm that the two signals are not identical.

-----

**<span style="color:#075a04">12. Using a third scope, probe the voltage differences between the two bases (connect each pin of the scope to one of the bases). Since the waveforms are not identical, this signal will not be a zero signal. The asymmetry in the design is being compensated by a longer PWM pulse on one motor compared to the other. Using the waveform on the third scope, measure  the duration difference of the  two PWM inputs. Click on the third scope and change the scope's timespan to 0.25ms/division in order to accurately measure the duration of the pulse. Enter your answer in units of us, and only enter numerical values (no units). </span>**

< YOUR ANSWER ON GRADESCOPE >

<a id='part2'></a>
# <span style="color:blue">Part 2: Closed-Loop Control</span>

<a id='part21'></a>
## Closed-Loop Design

Given the data you found in System ID, we know that the car does not match perfectly with our linear model *(think about why we know this)*. In order to correct for the imperfect model, we can build a closed loop controller that uses feedback to ensure that both wheels still drive at the same velocity. The controller has 2 inputs, namely the desired velocity and the difference between the wheel distances $\delta[i] = d_L[i] - d_R[i]$.

### 1. Please make sure you have read the Part 2 of [the lab note](https://drive.google.com/file/d/1zOL1_qXV2KM-Sc727ymCfZUzt4oqIgoE/view?usp=sharing) before you begin. You may also find it helpful to read [this PDF](https://inst.eecs.berkeley.edu/~ee16b/fa17/proj/controls-primer.pdf) for the derivation of the closed-loop system and its eigenvalue.

<span style='color:red'>**2. Using the information above, draw a block diagram of both wheels' controllers.**</span> Do not discard this diagram. Keep this somewhere, as you may use it later in your next labs, or report.

<span style='color:red'>**3. Now derive the closed loop update equation for the differences $\delta[n]$ corresponding to your controller.**</span> Keep this somewhere, as you will use it in your report.

Now that we've derived the model, we can begin implementing it. Fill in the two functions below.

In [ ]:
# DESIGN YOUR CLOSED-LOOP CONTROL HERE USING f_left AND f_right AS YOUR FEEDBACK TERMS
def drive_straight_left_cl(v_star, delta):
    """Returns the closed loop input 'u' for the left wheel"""
    # YOUR CODE HERE
    return  # u

def drive_straight_right_cl(v_star, delta):
    """Returns the closed loop input 'u' for the right wheel"""
    # YOUR CODE HERE
    return  # u

<a id='part22'></a>
## Closed Loop Simulation and feedback Gain (f-value) Selection

Next, we need to choose values for the control gains so that the $\delta[i]$ update equation is stable. *Concept check: For what eigenvalues is a discrete system stable?* There should be a range of parameter values that make it stable, so we can tune the gains to get the desired performance.

Take a look at each group of perfect/noiseless-mismatched-model/noisy-mismatched-model plots below for each feedback gain f-value pair and relate how our choice in feedback gain (f) affects the car's performance. For each pair of plots, ask yourself these questions:
1. Does the car seem to drive straight in the perfectly matched model?
2. Does the car seem to correct for any errors that arise from a mismatched model? Does it end up driving straight?
3. How well does the car correct for errors?
3. Why do the chosen feedback gain f-values make our system exhibit such behaviors?

<span style='color:red'>**Try the 5 different sets of gains below and see the effect of each.**</span>

In [ ]:
f_left, f_right = None, None
steady_state_error = None

# Maps description of f_values to (f_left, f_right)
f_values = {
    "marginally stable": (-0.3, 0.3),
    "stable, corrects error in one step": (0.5, 0.5),
    "oscillatory, marginally stable": (1, 1),
    "oscillatory, unstable": (1.1, 1.1),
    "stable": (0.15, 0.15)
}

for key in f_values:
    f_left, f_right = f_values[key]

    titles = ["Closed-loop control with perfect model",
              "Closed-loop control with model mismatch, fL={}, fR={}".format(f_left, f_right),
             "Closed-loop control with model mismatch + noise, fL={}, fR={}".format(f_left, f_right)]
    utils.two_sims(titles, simulator, v_star, drive_straight_left_cl, drive_straight_right_cl,sim_length=sim_length)
    utils.plot_car_traj(titles, simulator, v_star, drive_straight_left_cl, drive_straight_right_cl,mismatch_error_arr=[0.0, mismatch_error, mismatch_error],sim_length=sim_length, u_noise_factor_arr=[0, 0, 0.05])

    print("fL={}, fR={}".format(f_left, f_right))
    print("Eigenvalue of system: 1-fL-fR={:.2g}; {}".format(1-f_left-f_right, key))
    plt.show()

**<span style="color:#075a04">13. What is the steady-state error for the non-oscillatory marginally stable case when there is no mismatch in the model? Enter the numerical value as displayed in the script. </span>**

< YOUR ANSWER ON GRADESCOPE >

**<span style="color:#075a04">14. What is the steady-state error for the "stable, corrects error in one step" case when mismatch is included in the model? Enter the numerical value as displayed in the script. </span>** 

< YOUR ANSWER ON GRADESCOPE >

**<span style="color:#075a04">15. What is the steady-state error for the non-oscillatory "stable" case when mismatch is included in the model? Enter the numerical value as displayed in the script. </span>** 

< YOUR ANSWER ON GRADESCOPE >

<a id='part23'></a>
## Closed-Loop Implementation

Now implement the controller on the Arduino. The sketch **`closed_loop.ino`** contains some code that implements the closed loop control. 

- Fill out `CODE BLOCK CON1` with the control gains **`f_left`** and **`f_right`** for the "stable" case as well as your theta, beta, and v_star values from System ID.
- Fill out `CODE BLOCK CON2` with your closed-loop control scheme.
- Fill out `CODE BLOCK CON0` in the body of the loop function to calculate the control inputs. Inspect this code to see how the control system is implemented.
- Upload the sketch to your Arduino.


**<span style="color:#075a04">16. Disconnect the oscilloscopes, run the code in TinkerCAD, and observe the output in the serial monitor. How many steps does it take for the control system to correct the initial mismatch in positions ($|\delta|<3$)? e.g., If the sequence printed in the serial monitor is -10, 8, 2, -1, 0, 1, 2,..., the answer would be 3. Only enter the numerical value which should be an integer (e.g. 3). This simulation could take 10-15 seconds, so please be patient.</span>** 

< YOUR ANSWER ON GRADESCOPE >

<a id='part24'></a>
## Feedback Gain (f-value) Tuning

In order to find the "right" f values, we will need to analyze the behavior of our car. Recall in lecture that we learned about how the control values will affect the convergence of a system.


Figure 1 to Figure 6 in the [the lab note](https://drive.google.com/file/d/1zOL1_qXV2KM-Sc727ymCfZUzt4oqIgoE/view?usp=sharing) can be summarized as:
- if f is too small, it takes a long time to converge to our desired solution
- if f is too big, oscillations happen

We can use this information to help guide us to f values that get our car going straight. Check out [this youtube playlist](https://www.youtube.com/playlist?list=PLQ1OVCqw6RUPgCmv8H6y9qbcMBT8lotDt) if you need some examples of what each case could look like (note that the videos in the playlist call them "k-values" instead of "f-values").

Use the script below to answer the following 2 questions.


In [ ]:
f_left, f_right = None, None
steady_state_error = None

# Maps description of f_values to (f_left, f_right)
f_values = {
    "chosen f values": (0.8, 0.8)  # TODO: record your chosen f-values here
}

for key in f_values:
    f_left, f_right = f_values[key]

    titles = ["Closed-loop control with perfect model",
              "Closed-loop control with model mismatch, fL={}, fR={}".format(f_left, f_right),
             "Closed-loop control with model mismatch + noise, fL={}, fR={}".format(f_left, f_right)]
    utils.two_sims(titles, simulator, v_star, drive_straight_left_cl, drive_straight_right_cl,sim_length=sim_length)
    utils.plot_car_traj(titles, simulator, v_star, drive_straight_left_cl, drive_straight_right_cl,mismatch_error_arr=[0, mismatch_error, mismatch_error],sim_length=sim_length, u_noise_factor_arr=[0, 0, 0.05])

    print("fL={}, fR={}".format(f_left, f_right))
    print("Eigenvalue of system: 1-fL-fR={:.2g}; {}".format(1-f_left-f_right, key))
    plt.show()

**<span style="color:#075a04">17. In the Python code space provided above, set both f values to 0.8 and observe the car's behavior and it's wheel distances ($\delta$). What is the steady state error in this case for a model that only includes mismatch? Only enter the numerical value with two decimal places as displayed by the script. </span>** 

< YOUR ANSWER ON GRADESCOPE >

<a id='part25'></a>

## Steady State Error Correction

Lastly, what if we want the car to travel in (approximately) the same direction as when it starts moving (along the x-axis in this case)? We can manually add the known steady state error $\delta_{ss}$ to each calculation of the error $\delta$. Accounting for the known $\delta_{ss}$ helps the model better respond to unexpected pertubations affecting $\delta$ which increases the model's control strength.

We now calculate $\delta[i]=d_L - d_R + \delta_{ss}$.


In [ ]:
f_left, f_right = [0.3, 0.3]
title = ["Closed-loop control with perfect model",
        ("Closed loop control with model mismatch and\nsteady state error correction, fL={}, fR={}").format(f_left, f_right),
         ("Closed loop control with model mismatch+noise and\nsteady state error correction, fL={}, fR={}").format(f_left, f_right)]

# Find steady_state_error
_, steady_state_error = simulator.simulate(v_star, drive_straight_left_cl, drive_straight_right_cl, mismatch_error=mismatch_error)

# Simulate with steady_state_error
d, _ = simulator.simulate(v_star, drive_straight_left_cl, drive_straight_right_cl, mismatch_error=mismatch_error, offset=steady_state_error)
simulator.plot(d, title[1])
utils.plot_car_traj(title, simulator, v_star, drive_straight_left_cl, drive_straight_right_cl, offset=steady_state_error, mismatch_error_arr=[0, mismatch_error, mismatch_error],sim_length=sim_length, u_noise_factor_arr=[0, 0, 0.05])



**<span style="color:#075a04">18. (Set both f values to 0.3!) After the correction, what is the final steady-state error for a noiseless system with 5% mismatch? Only enter the numerical value with two decimal places as displayed by the script. </span>** 

< YOUR ANSWER ON GRADESCOPE >

#### <span style="color:#075a04"> 19. Please upload two screenshots of your final Tinkercad circuits, including both the circuit and schematic views, to Gradescope.
    

<a id='part4'></a>
# <span style="color:#ba190f">CHECKOFF</span>
-----

Submit your answers to Questions 1-18 to the Gradescope assignment "Lab 7: Controls Part 1." Submit your screenshots for Question 19 to the Gradescope assignment "Lab 7: Tinkercad Circuits." Lab 7 is due on Sunday, July 24 at 11:59pm. No late submissions will be accepted.

### Make sure your circuits are saved properly in Tinkercad. You will need them in the next project phase!

### Remember what each part of your circuit is for by recording this information in a Google doc or somewhere else safe. You will need to write a summary for your final lab report.

### Save this notebook somewhere you can access!

We recommend making a shared Google Drive with your lab partner, since GitHub can be annoying with Jupyter notebooks. It's good to familiarize yourself with the user interface of Tinkercad and understand the basic functions of these circuits that you have built and analyzed.


### Great job finishing 16B Lab 7!
